# Road Following 

collision avoidanceを実行していれば、今回実行する次の3ステップには馴染みがあるでしょう。


1. Data収集
2. 学習
3. デプロイ

このnotebookでも、同じように実行していきます。ただし、今回はJetBotが道路（または実際に任意のパスまたはターゲットポイント）を走行できるようにするために、画像分類の代わりに、**回帰**という別の手法で学習させます。

 1. JetBotをパス上のさまざまな位置に配置します（中心からのオフセット、さまざまな角度など）

> データの多様性こそが重要であると、collision avoidanceのときに覚えたはずです。


2. robotからのライブカメラの画像を画面に表示します
3. マウスでX,Y座標を移動することで、`greenの点`をロボットが移動したい方向に移動します。
4. この`greenの点`のX、Y座標の値をロボットのカメラからの画像と、一緒に保存します


notebookの学習で、ラベルのX, Yの値の推論をおこなうためにneural networkで学習します。このライブデモでは、Jetbotのステアリングの値を計算するために予測されたX、Y値を使用します。(角度として「正確に」ではありません。画像のキャリブレーションが必要になりますが、角度にほぼ比例するため、制御は正常に機能します)

それでは、このサンプルの目標点は、どこに配置するのがいいでしょうか？ 下記が役立つと思われるガイドです。

1. カメラからのライブデモフィードを見ます
2. ロボットがたどるべき経路を想像してください（道路からの脱輪などを回避するために必要な距離を計算してみてください）
3. ロボットが道路を「脱輪」する事なく目標点にまっすぐ進むことができるように、目標点はこの経路に沿ったできるだけ遠くの位置に配置します。

> 例として、もし真っ直ぐのロードがあったとします、目標点は地平線にします。もし、鋭く曲がったカーブなら、脱輪しないレベルで、ロボットの近くに配置する必要があるでしょう。
 
ディープラーニングのモデルが意図したとおりに機能すると仮定すると、これらのラベリングガイドラインでは次のことが保証されます。

1.ロボットは、ターゲットに向かって安全に直接移動できます（範囲外に出ることなく）。
2.目標点は、想像した経路に沿って継続的に処理されます。

この処理で得られるものは、望む軌道に沿って動く「スティック上のニンジン」です。ディープラーニングがニンジンを配置する場所を推論し、JetBotがそれに追従します。

### Labeling example video(ラベリングのデモ動画)

下記URLの動画を参考に、画像にラベルを付ける方法の例を確認します。このモデルはたった123枚の画像で動作しました:)

YouTubeのサンプル動画

https://www.youtube.com/embed/FW4En6LejhI


### Import Libraries(ライブラリのimport)

それでは、"data collection"のために必要なライブラリ全部をimportして、作業を開始します。ラベルと紐付いた画像を表示し、保存するためにOpenCVを使用します。uuid、datetimeなどのライブラリは、イメージのファイル名の命名に使用されます。

In [ ]:
# IPython Libraries for display and widgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg

# Python basic pakcages for image annotation
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time

### Display Live Camera Feed(ライブカメラフィードの表示)

最初に、teleperation notebookで実行したようにカメラの初期化と表示をおこないます。

JetBotのカメラクラスは、CSI MIPI cameraを有効にするために使います。neural networkでは、224x224ピクセスの画像データをインプットとして使います。データセットのファイルサイズを最小化するために、カメラをそのサイズに設定します(このタスクのために動作する動作する事は確認しています) いくつかのシナリオでは、画像サイズを大きくし収集し、後で目的のサイズに縮小する方がいいかもしれません。

本サンプルを実行するにあたり、まずnvargus-daemon(カメラ等で使用)をリスタートします。

In [ ]:
!echo jetbot | sudo -S systemctl restart nvargus-daemon

In [ ]:
camera = Camera.instance(width=224, height=224, fps=2)

image_widget = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jpeg', width=224, height=224)

x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x')
y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y')

def display_xy(camera_image):
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * 224 / 2 + 112)
    y = int(y * 224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)

display(widgets.HBox([image_widget, target_widget]), x_slider, y_slider)

### Collect data(データ収集)

次のコードブロックは、ライブ画像フィードと保存した画像の数を表示します。ターゲットとなるX、Y値の値を下記のように保存します。


1. 目標点に緑のドットを移動します。
2. 保存ボタンを押します(本サンプル用に、オフィシャルのサンプルを修正しています。)


これで、``dataset_xy``フォルダーに、

``xy_<x value>_<y value>_<uuid>.jpg``

の形式で保存されます。


学習時に、ファイル名からx,yの値をパースし、イメージとともに読み込みます。

In [ ]:
DATASET_DIR = 'dataset_xy'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

#for b in controller.buttons:
#    b.unobserve_all()

count_widget = widgets.IntText(description='count', value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))

def xy_uuid(x, y):
    return 'xy_%03d_%03d_%s' % (x * 50 + 50, y * 50 + 50, uuid1())

def save_snapshot():
    #if change['new']:
    uuid = xy_uuid(x_slider.value, y_slider.value)
    image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image_widget.value)
        count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

#controller.buttons[13].observe(save_snapshot, names='value')
button_layout = widgets.Layout(width='128px', height='64px')
add_button = widgets.Button(description='追加', button_style='success', layout=button_layout)

display(widgets.VBox([
    target_widget,
    x_slider,
    y_slider,
    count_widget,
    add_button
]))

add_button.on_click(lambda x: save_snapshot())

#display(widgets.HBox([image_widget, target_widget]), x_slider, y_slider)

## Next(次)

次は、train_mlodel.ipynbで学習をおこないます。